In [10]:
import pandas as pd  

def convert_excel_to_conf(input_excel, output_file):  
    # Đọc dữ liệu từ file Excel  
    df = pd.read_excel(input_excel)  

    # Tạo danh sách để lưu các dòng sẽ ghi vào file  
    converted_lines = ['[Service]\n']  

    # Duyệt qua từng dòng trong DataFrame  
    for i in range(len(df)):  
        parameter = df.iloc[i]['Parameter']  
        setup_value = df.iloc[i]['Setup Value']  

        # Kiểm tra kiểu dữ liệu và xử lý  
        if isinstance(parameter, str):  
            parameter = parameter.strip()  
        else:  
            parameter = ''  

        if isinstance(setup_value, str):  
            setup_value = setup_value.strip()  
        else:  
            setup_value = ''  
        
        # Xử lý theo từng kiểu tham số  
        if parameter in ["Environment", "EnvironmentFile"]:  
            # Không thêm dấu ngoặc kép cho dòng EnvironmentFile nếu không có giá trị  
            converted_lines.append(f'{parameter}={setup_value}\n')  
            # # Thêm ghi chú cho EnvironmentFile  
            # converted_lines.append(f'# This is a file that the user can use for overrides of the kubelet args as a last resort. Preferably,\n')  
            # converted_lines.append(f'# the user should use the .NodeRegistration.KubeletExtraArgs object in the configuration files instead.\n')  
            # converted_lines.append(f'# KUBELET_EXTRA_ARGS should be sourced from this file.\n')  
        elif parameter == "ExecStart" and setup_value == "":  
            # Thêm dòng trống cho ExecStart  
            converted_lines.append(f'{parameter}=\n')  
        elif parameter == "ExecStart":  
            # Đối với ExecStart có giá trị, không bao quanh bằng dấu ngoặc kép  
            converted_lines.append(f'{parameter}={setup_value}\n')  
        else:  
            converted_lines.append(f'{parameter}="{setup_value}"\n')  
    
    # # Thêm ghi chú cho dòng EnvironmentFile đầu tiên  
    # converted_lines.insert(5, '# This is a file that "kubeadm init" and "kubeadm join" generate at runtime, populating\n')  
    # converted_lines.insert(6, '# the KUBELET_KUBEADM_ARGS variable dynamically\n')  

    # Ghi vào file đầu ra dưới định dạng .conf  
    with open(output_file, 'w', encoding='utf-8') as outfile:  
        outfile.writelines(converted_lines)  


# Sử dụng hàm  
input_filename = '/workspaces/test/10cu/10-kubeadm.conf.xlsx'  # Tên file Excel đầu vào  
output_filename = 'output.conf'  # Tên file .conf đầu ra  
convert_excel_to_conf(input_filename, output_filename)  

print(f"Converted file saved as {output_filename}")  

Converted file saved as output.conf


In [11]:
import pandas as pd  
import json  
import configparser  
from deepdiff import DeepDiff  

# Đọc file .conf và chuyển thành dictionary  
def load_conf(file_path):  
    config = configparser.ConfigParser(allow_no_value=True, delimiters=('=', ':'))  
    config.optionxform = str  # Giữ nguyên chữ hoa chữ thường  
    
    # Đọc nội dung file .conf  
    with open(file_path, 'r') as file:  
        content = file.read()  
    
    # Thêm một section giả (section 'default') nếu file không có section header  
    if not content.startswith('['):  
        content = '[default]\n' + content  
    
    config.read_string(content)  # Đọc từ chuỗi đã chỉnh sửa  
    
    result = {}  
    for section in config.sections():  
        result[section] = {}  
        for option in config.options(section):  
            # Lưu trữ nhiều giá trị cho cùng một tùy chọn  
            if option in result[section]:  
                # Nếu đã tồn tại, đưa giá trị vào danh sách  
                if isinstance(result[section][option], list):  
                    result[section][option].append(config.get(section, option))  
                else:  
                    result[section][option] = [result[section][option], config.get(section, option)]  
            else:  
                result[section][option] = config.get(section, option)  

    return result  

# So sánh hai dictionary  
def compare_dic(data1, data2):  
    diff_result = DeepDiff(data1, data2, ignore_order=True)  # Sử dụng deepdiff để so sánh  
    return diff_result  

# Khởi tạo file gốc và file được tạo ra  
file1 = '/workspaces/test/10cu/10-kubeadm.conf'  # Đường dẫn đến file .conf gốc  
file2 = 'output.conf'  # File .conf được tạo ra từ mã trước  

# Tải về nội dung của hai file để so sánh  
data1 = load_conf(file1)  
data2 = load_conf(file2)  

# So sánh hai dictionary và lấy kết quả  
diff_result = compare_dic(data1, data2)  

# Lưu kết quả vào file JSON  
with open('diff_result_conf.json', 'w') as json_file:  
    json.dump(diff_result, json_file, indent=4)  

print("File diff_result_conf.json đã được lưu.")  

DuplicateOptionError: While reading from '<string>' [line  3]: option 'Environment' in section 'Service' already exists

In [7]:
import pandas as pd
import re
import json
import os
import configparser
from deepdiff import DeepDiff

# Đọc file .conf và chuyển thành dictionary
def load_conf(file_path):
    config = configparser.ConfigParser()
    
    # Đọc nội dung file .conf
    with open(file_path, 'r') as file:
        content = file.read()
    
    # Thêm một section giả (section 'default') nếu file không có section header
    if not content.startswith('['):
        content = '[default]\n' + content
    
    # Ghi nội dung có section header vào file tạm thời
    with open('temp.conf', 'w') as temp_file:
        temp_file.write(content)
    
    # Đọc file tạm thời và chuyển thành dictionary
    config.read('temp.conf')
    config_dict = {section: dict(config.items(section)) for section in config.sections()}
    
    return config_dict

# So sánh hai dictionary và trả về sự khác biệt dưới dạng JSON
def compare_dic(dict1, dict2):
    diff = DeepDiff(dict1, dict2, ignore_order=True)
    
    result = {}

    if not diff:
        result["message"] = "Same"
    else:
        result["message"] = "Different"
        
        # Nếu có sự thay đổi giá trị
        if 'values_changed' in diff:
            result["values_changed"] = []
            for key, change in diff['values_changed'].items():
                result["values_changed"].append({
                    "key": key,
                    "old_value": change['old_value'],
                    "new_value": change['new_value']
                })

        # Nếu có mục được thêm vào
        if 'dictionary_item_added' in diff:
            result["dictionary_item_added"] = []
            for key in diff['dictionary_item_added']:
                result["dictionary_item_added"].append({
                    "key": key,
                    # "added_in": "file 2"
                })

        # Nếu có mục bị xóa
        if 'dictionary_item_removed' in diff:
            result["dictionary_item_removed"] = []
            for key in diff['dictionary_item_removed']:
                result["dictionary_item_removed"].append({
                    "key": key,
                    # "removed_in": "file 2"
                })

    return result

# Đọc hai file .conf
file1 = '/workspaces/test/10cu/10-kubeadm.conf'
file2 = output_filename

data1 = load_conf(file1)
data2 = load_conf(file2)

# So sánh hai dictionary và lấy kết quả
diff_result = compare_dic(data1, data2)

# with open(sheet_name + "_compare.json", 'w') as json_file: 
#     json.dump(diff_result, json_file, indent=4)

# Lưu kết quả vào file JSON
with open('diff_result_conf.json', 'w') as json_file:
    json.dump(diff_result, json_file, indent=4)

# print("File diff_result.json đã được lưu.")

DuplicateOptionError: While reading from 'temp.conf' [line  3]: option 'environment' in section 'Service' already exists